In [ ]:
from hossam import load_data
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sb
import pandas as pd
import numpy as np
from pandas import DataFrame
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from scipy.stats import zscore, pearsonr, spearmanr, probplot, shapiro, kstest, anderson

from statsmodels.formula.api import ols
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [19]:
df = pd.read_excel('3사분위수_데이터.xlsx')

In [ ]:
myorigin = load_data('df')

print("\n===== 데이터 크기 확인 =====")
print(f"데이터셋 크기: {df.shape}")
print(f"열 개수: {df.shape[1]}")
print(f"행 개수: {df.shape[0]}")
print("\n===== 타입확인 =====")

print(df.info())

df.head()

df는 존재하지 않는 데이터에 대한 요청입니다.

===== 데이터 크기 확인 =====
데이터셋 크기: (13461, 11)
열 개수: 11
행 개수: 13461

===== 타입확인 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13461 entries, 0 to 13460
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  13461 non-null  int64  
 1   price       13461 non-null  int64  
 2   carat       13461 non-null  float64
 3   cut         13461 non-null  object 
 4   color       13461 non-null  object 
 5   clarity     13461 non-null  object 
 6   x           13461 non-null  float64
 7   y           13461 non-null  float64
 8   z           13461 non-null  float64
 9   depth       13461 non-null  float64
 10  table       13461 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 1.1+ MB
None


,Unnamed: 0,price,carat,cut,color,clarity,x,y,z,depth,table
0,90,2757,0.70,Ideal,E,SI1,5.70,5.72,3.57,62.5,57.0
1,91,2757,0.86,Fair,E,SI2,6.45,6.33,3.52,55.1,69.0
2,92,2757,0.70,Ideal,G,VS2,5.70,5.67,3.50,61.6,56.0
3,93,2759,0.71,Very Good,E,VS2,5.68,5.73,3.56,62.4,57.0
4,94,2759,0.78,Very Good,G,SI2,5.81,5.85,3.72,63.8,56.0


In [8]:
cv = ['carat', 'depth', 'table', 'x', 'y', 'z']
X_num = df[cv].copy()


In [9]:
def calculate_vif(X):
    X = sm.add_constant(X)
    vif = pd.DataFrame()
    vif['variable'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i)
                  for i in range(X.shape[1])]
    return vif


변수들이 서로 독립적으로 정보 제공

회귀계수의 불안정성 위험 낮음

In [10]:
while True:
    vif = calculate_vif(X_num)
    vif = vif[vif['variable'] != 'const']

    max_vif = vif['VIF'].max()

    if max_vif > 10:
        drop_var = vif.sort_values('VIF', ascending=False).iloc[0]['variable']
        print(f'Drop: {drop_var}, VIF={max_vif:.2f}')
        X_num = X_num.drop(columns=[drop_var])
    else:
        break


Drop: y, VIF=34.99
Drop: x, VIF=25.55


In [11]:
X_num.columns


Index(['carat', 'depth', 'table', 'z'], dtype='object')

In [12]:
cat_cols = ['cut', 'color', 'clarity']
X_cat = pd.get_dummies(df[cat_cols], drop_first=True)


In [13]:
X_final = pd.concat([X_num, X_cat], axis=1)


In [14]:
X_final = X_final.astype(float)


In [15]:
y = df['price']

X_final = sm.add_constant(X_final)
model = sm.OLS(y, X_final).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     2314.
Date:                Tue, 13 Jan 2026   Prob (F-statistic):               0.00
Time:                        16:25:32   Log-Likelihood:                -99661.
No. Observations:               13461   AIC:                         1.994e+05
Df Residuals:                   13439   BIC:                         1.995e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -2875.8403    231.424    -12.427

In [16]:
final_vif = calculate_vif(X_final.drop(columns='const'))
final_vif


,variable,VIF
0,const,4559.271284
1,carat,8.355860
2,depth,1.677015
3,table,1.660956
4,z,7.568925
5,cut_Good,3.238920
6,cut_Ideal,6.940654
7,cut_Premium,5.526408
8,cut_Very Good,5.249706
9,color_E,2.008307


In [ ]:
final_vif = calculate_vif(X_final.drop(columns='const'))
final_vif.sort_values('VIF', ascending=False)


,variable,VIF
0,const,4559.271284
16,clarity_SI1,10.899006
17,clarity_SI2,10.265893
1,carat,8.355860
19,clarity_VS2,7.938905
4,z,7.568925
6,cut_Ideal,6.940654
18,clarity_VS1,6.058477
7,cut_Premium,5.526408
8,cut_Very Good,5.249706


1. 연속형 변수만 선택
2. VIF 계산
4. 남은 연속형 고정
5. 명목형 더미화
6. 결합
7. 회귀
명목형 변수를 더미화할 경우 다중공선성 진단이 복잡해질 수 있어,
연속형 변수에 대해 먼저 VIF를 통해 구조적 공선성을 점검한 후
안정화된 모형에 명목형 변수를 추가하였다.